In [1]:
#Script para identificar determinantes de citações na base USPTO
#Renato Kogeyama

#update December 29, 2019
#including ancestry citation

#10-03-2019
#I rewrote the citation data to clean the strings


#09-27-2019
#I am renaming citation as forward citation and backward citation

#09-17-2018

#Alto uso de memoria - rodar no Amazon AWS 

import pandas as pd
# from sklearn.preprocessing import MinMaxScaler #To normalize data

In [2]:
#ler o arquivo de citacoes
# asg = pd.read_csv(r'/home/rkogeyam/PATENT_CITATION/fwd_back_cit.csv', index_col=0, dtype={0: object, "forward_citation": float,"backward_citation": float})
src = '/home/rkogeyam/PATENT_CITATION/cit_tree.csv'
asg = pd.read_csv(src, index_col=1)


/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
asg.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
asg.head()

,back_citation,parent_back_citation
citation_id,,
0000000,1,10.0
000004,1,1.0
000006,1,11.0
0000482,1,30.0
0000H94,1,14.0


In [5]:
asg.tail()

,back_citation,parent_back_citation
citation_id,,
l640855,1,6.0
l92594,1,2.0
pctus9926473,1,42.0
pp6708,1,0.0
re25727,1,68.0


In [6]:
asg.describe()

,back_citation,parent_back_citation
count,7.049261e+06,7.049261e+06
mean,1.297236e+01,2.194859e+02
std,3.053004e+01,1.498394e+03
min,1.000000e+00,0.000000e+00
25%,2.000000e+00,5.000000e+00
50%,5.000000e+00,2.600000e+01
75%,1.300000e+01,1.080000e+02
max,3.922000e+03,2.274790e+05


In [7]:
% matplotlib inline

In [8]:
# df = pd.read_csv('patent.csv', sep='\t', error_bad_lines=False, na_values='-', usecols=[0,1,4,5,6,7,8])
# df = pd.read_csv('patent.csv', sep='\t', na_values='-', usecols=[0,1,4,5,6,7,8])
df = pd.read_csv('patent.csv',sep='\t', error_bad_lines=False)
#o problema aqui eh que estas linhas tem tags tipo <b><\b>
#poderia limpa-las antes de 

b'Skipping line 4232501: expected 10 fields, saw 11\n'
b'Skipping line 4267898: expected 10 fields, saw 11\n'
b'Skipping line 4327680: expected 10 fields, saw 11\nSkipping line 4368028: expected 10 fields, saw 11\nSkipping line 4377090: expected 10 fields, saw 11\n'
b'Skipping line 4426415: expected 10 fields, saw 11\n'
/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.shape

(6488261, 10)

In [10]:
df.sort_values(by=['date'])
# df

,id,type,number,country,date,abstract,title,kind,num_claims,filename
0,3930271,utility,3930271,US,1976-01-06,A golf glove is disclosed having an extra fin...,Golf glove,A,4.0,pftaps19760106_wk01.zip
924,3931195,utility,3931195,US,1976-01-06,The compounds are of the heterocyclic class o...,Substituted piperidines,A,28.0,pftaps19760106_wk01.zip
923,3931194,utility,3931194,US,1976-01-06,Compounds of the formula ##SPC1## Wherein R i...,"2-Heteroaryl-methyl)-5,9.beta.-dialkyl-6,7-ben...",A,6.0,pftaps19760106_wk01.zip
922,3931193,utility,3931193,US,1976-01-06,"Quinine, quinidine and analogs thereof, are p...",Quinuclidine derivatives,A,5.0,pftaps19760106_wk01.zip
921,3931192,utility,3931192,US,1976-01-06,"Quinine, quinidine and analogs thereof, are p...",Process for piperidine intermediates for quini...,A,2.0,pftaps19760106_wk01.zip
920,3931191,utility,3931191,US,1976-01-06,"A process of converting 2,3,4,5-tetrahydropyr...",Conversion of tetrahydropyrimidines to pyridines,A,5.0,pftaps19760106_wk01.zip
919,3931190,utility,3931190,US,1976-01-06,A process is provided for the preparation of ...,Preparation of 2-chloropyridine by hydrogenolysis,A,9.0,pftaps19760106_wk01.zip
918,3931189,utility,3931189,US,1976-01-06,Compounds of the formula ##SPC1## Wherein PA1...,"N-(heteroaryl-methyl)-6,14-(endoethano or endo...",A,6.0,pftaps19760106_wk01.zip
917,3931188,utility,3931188,US,1976-01-06,Compounds having the formula ##SPC1## In whic...,"3-Hydroxy-5,6-benzomorphinan derivatives",A,9.0,pftaps19760106_wk01.zip
916,3931187,utility,3931187,US,1976-01-06,Compounds of the formula ##SPC1## Wherein PA1...,"N-(heteroaryl-methyl)-7.alpha.-acyl-6,14-(endo...",A,6.0,pftaps19760106_wk01.zip


In [11]:
df['date']=pd.to_datetime(df['date'], format="%Y-%m-%d", errors='coerce') #conversao de string para data

In [12]:
# dados desnecessarios
# df.drop(columns=['date'])

# df=df.dropna()
#dropando dados que nao sao importantes no momento
#a coluna country eh toda US. filename nao tem uso agora, e a coluna number eh identica a coluna id.
# df=df.drop(columns=['number', 'filename', 'country']) 

In [13]:
# df.plot.hist()

In [14]:
# juntando as citacoes a tabela principal
# merge left assim citacoes que nao encontram patentes sao descartadas
df=pd.merge(df, asg, how='left', left_on='id', right_index=True)

In [15]:
#df.sort_values('citation_id').dropna().tail()

In [16]:
df.head()

,id,type,number,country,date,abstract,title,kind,num_claims,filename,back_citation,parent_back_citation
0,3930271,utility,3930271,US,1976-01-06,A golf glove is disclosed having an extra fin...,Golf glove,A,4.0,pftaps19760106_wk01.zip,NaN,NaN
1,3930272,utility,3930272,US,1976-01-06,A lock for a height-adjustable crib or plaype...,Crib leg lock,A,3.0,pftaps19760106_wk01.zip,NaN,NaN
2,3930273,utility,3930273,US,1976-01-06,A bed safety side rail arrangement which incl...,Bed safety side rail arrangement,A,24.0,pftaps19760106_wk01.zip,NaN,NaN
3,3930274,utility,3930274,US,1976-01-06,The assembly includes a longitudinal axis and...,Assembly for use in recreational activities,A,7.0,pftaps19760106_wk01.zip,NaN,NaN
4,3930275,utility,3930275,US,1976-01-06,A novel slipper and its method of fabrication...,Method of fabricating a slipper,A,9.0,pftaps19760106_wk01.zip,NaN,NaN


In [17]:
# NaN in citations means a patent did not receive citations
# df['forward_citation']=df['forward_citation'].fillna(value=0) 
df['back_citation']=df['back_citation'].fillna(value=0) #NA eh zero mesmo pq a patente nao recebeu citacoes
df['parent_back_citation']=df['parent_back_citation'].fillna(value=0) #NA eh zero mesmo pq a patente nao recebeu citacoes

In [18]:
#df=df.dropna()

In [19]:
#estamos prontos para testar o primeiro modelo 

In [20]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df.head()

,id,type,number,country,date,abstract,title,kind,num_claims,filename,back_citation,parent_back_citation,year,month,day
0,3930271,utility,3930271,US,1976-01-06,A golf glove is disclosed having an extra fin...,Golf glove,A,4.0,pftaps19760106_wk01.zip,0.0,0.0,1976.0,1.0,6.0
1,3930272,utility,3930272,US,1976-01-06,A lock for a height-adjustable crib or plaype...,Crib leg lock,A,3.0,pftaps19760106_wk01.zip,0.0,0.0,1976.0,1.0,6.0
2,3930273,utility,3930273,US,1976-01-06,A bed safety side rail arrangement which incl...,Bed safety side rail arrangement,A,24.0,pftaps19760106_wk01.zip,0.0,0.0,1976.0,1.0,6.0
3,3930274,utility,3930274,US,1976-01-06,The assembly includes a longitudinal axis and...,Assembly for use in recreational activities,A,7.0,pftaps19760106_wk01.zip,0.0,0.0,1976.0,1.0,6.0
4,3930275,utility,3930275,US,1976-01-06,A novel slipper and its method of fabrication...,Method of fabricating a slipper,A,9.0,pftaps19760106_wk01.zip,0.0,0.0,1976.0,1.0,6.0


In [21]:
df.shape

(6488261, 15)

In [22]:
df.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,num_claims,back_citation,parent_back_citation,year,month,day
count,6.488254e+06,6.488261e+06,6.488261e+06,6.488256e+06,6.488256e+06,6.488256e+06
mean,1.414322e+01,7.257130e+00,1.005466e+02,2.002723e+03,6.590933e+00,1.564227e+01
std,1.200423e+01,2.569204e+01,1.066787e+03,1.132268e+01,3.402092e+00,8.827004e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,1.976000e+03,1.000000e+00,1.000000e+00
25%,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN
50%,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN
75%,NaN,5.000000e+00,1.300000e+01,NaN,NaN,NaN
max,8.870000e+02,3.433000e+03,1.567150e+05,2.017000e+03,1.200000e+01,3.100000e+01


In [23]:
# 19-09-26
# este eh o trecho que deve ser alterado
# ao inves de gerar varias dummies, vou categorizar a variavel

# df=pd.get_dummies(df, columns=['type', 'kind'])
# df.describe()

df["type"] = df["type"].astype('category')
df["kind"] = df["kind"].astype('category')


#eh um bom ponto para salvar o arquivo
# df.to_csv('df_w_pat_cit.csv') #versao original

# df.to_csv('df_w_pat_cit_190926.csv') #alteracao de type e kind para categoria, divisao de date em ano, mes e dia

df.to_csv('df_w_pat_cit_191229.csv') #eliminacao de number, filename e country

#comecar a partir dele em um novo notebook